# Apêndice H
---
### TCC Lucas Seixas Dias
### Análise Tarifária das Unidades Consumidoras da Universidade Federal da Bahia (UFBA)
### 12 de dezembro de 2022
---

### Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Ignorar mensagens de aviso do phyton
import warnings
warnings.filterwarnings('ignore') # Ignorar todos os avisos

### Configurando a visualização dos conjuntos de dados

In [ ]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)

---

### Importando planilhas

### Tarifas Coelba A4

In [ ]:
tarifa_a4 = pd.read_excel('dados/TARIFA_A4_COELBA_04-2022.xlsx', sheet_name=0)

In [ ]:
tarifa_a4 = tarifa_a4.set_index('ENQUADRAMENTO')

In [ ]:
tarifa_a4

### Tarifas Coelba B3

In [ ]:
tarifa_b3 = pd.read_excel('dados/TARIFA_B3_COELBA_04-2022.xlsx', sheet_name=0)

In [ ]:
tarifa_b3 = tarifa_b3.set_index('ENQUADRAMENTO')

In [ ]:
tarifa_b3

### Faturas UFBA

In [ ]:
df = pd.read_excel('dados/Faturamento_UFBA_2022.xlsx', sheet_name=0)

### Transformando "Mês/ano" em índice

In [ ]:
# Transformar de DataTime para String
df['Mês/ano'] = df['Mês/ano'].astype(str)

In [ ]:
# Deixar somente mês e ano
df['Mês/ano'] = df['Mês/ano'].str.extract(r'(\d{4}-\d{2})')

In [ ]:
df = df.set_index('Mês/ano')

### Preenchendo campos ausentes com zero

In [ ]:
df.fillna(value=0, inplace=True)

### Verificando valores perdidos com multas e juros

In [ ]:
df_leitura = df['Multa (R$)'] + df['Juros (R$)']
print('Por todo o histórico (jan2017 a set2022):')
print(round(df_leitura.sum(), 2))
print()

df_leitura = df_leitura[df_leitura.index >= '2020-08']
print('Nos últimos 24 meses:')
print(round(df_leitura.sum(), 2))
print()

df_leitura = df_leitura[df_leitura.index >= '2022-01']
print('No ano de 2022 (janeiro a setembro):')
print(round(df_leitura.sum(), 2))

### Ordenando conjunto de dados

In [ ]:
df.sort_values(by = ['Mês/ano', 'Consumos Ativos (KWh)'], ascending=[True,False], inplace = True)

### Filtrando apenas contas mensais

In [ ]:
df = df[df['Tipo Docmto.'] == 'Periodico']

### Verificando o conjunto de dados sem limitar a pesquisa de contratos

In [ ]:
len(df['Cta Ctto'].drop_duplicates())

In [ ]:
len(df['Unidade/UFBA'].drop_duplicates())

### Verificando quantidade e quais as unidades da UFBA no arquivo

In [ ]:
df['Unidade/UFBA'].describe()

---

### Valor dos tributos

In [ ]:
# 2022.
icms = 0.18
tributos = (1-icms)
tributos

# pis = 0.0099
# cofins = 0.0458
# tributos = (1-(icms+pis+cofins))

---

### Criando lista de unidades consumidoras que possuem conta ativa em set 2022

In [ ]:
df_set22 = df[df.index == '2022-09']

In [ ]:
len(df_set22['Cta Ctto'].drop_duplicates())

In [ ]:
unidades_set22 = df_set22['Unidade/UFBA']
unidades_set22 = pd.Series.tolist(unidades_set22)
len(unidades_set22)

In [ ]:
unidades_set22_grupo_a = df_set22[df_set22['Classe Cálculo'] == 'Grupo A']['Unidade/UFBA']
unidades_set22_grupo_a = pd.Series.tolist(unidades_set22_grupo_a)
len(unidades_set22_grupo_a)

In [ ]:
unidades_set22_grupo_b = df_set22[df_set22['Classe Cálculo'] == 'Grupo B']['Unidade/UFBA']
unidades_set22_grupo_b = pd.Series.tolist(unidades_set22_grupo_b)
len(unidades_set22_grupo_b)

### Criando lista das unidades consumidoras do Grupo A com nenhuma demanda de registrada maior que 75 kW desde 2019

In [ ]:
unidades_set22_grupo_a_75menos = []

for i in unidades_set22_grupo_a:
    
    df_leitura = df[df['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= '2019-01']
    
    
    if (df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'].max() < 75) \
    and (df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'].max() < 75):
        unidades_set22_grupo_a_75menos.append(i)

len(unidades_set22_grupo_a_75menos)

---

### Verificando se alguma unidade consumidora já esteve no Grupo A e hoje é Grupo B

In [ ]:
todas_unidades = df['Unidade/UFBA'].drop_duplicates()

for unid in todas_unidades:
    df_unid = df[df['Unidade/UFBA'] == unid]
    
    if df_unid['Classe Cálculo'].value_counts(normalize=True)[0] != 1:
        print()
        print(unid)

---

### Criando função que registra o maior valor entre Demanda Fora Ponta e Demanda Ponta de cada mês

In [ ]:
# Adicionei essa coluna para que a média móvel seja calculada com a maior demanda do mês. 

def maior(df):  
    if df['Demandas Ativas Registradas Fora de Ponta -  (KW)'] > df['Demandas Ativas Registradas Na Ponta -  (KW)']:
        return df['Demandas Ativas Registradas Fora de Ponta -  (KW)']
    else:
        return df['Demandas Ativas Registradas Na Ponta -  (KW)']

### Adicionando a coluna "Maior Demanda Entre Ponta e Fora Ponta" no conjunto de dados

In [ ]:
df['Maior Demanda Entre Ponta e Fora Ponta'] = df.apply(lambda df: maior(df), axis=1)

---

### Definindo novo conjunto de dados filtrando colunas

In [ ]:
df_cortado = df[['Classe Cálculo', 'Unidade/UFBA', 'Consumos Ativos (KWh)', 'Consumos Ativos Na Ponta (KWh)', \
                 'Consumos Ativos Fora de Ponta (KWh)', 'Consumos Reativos (KWh)', 'Demdas. Ativas Contdas. (DC)', \
                 'Demandas Ativas Registradas Na Ponta -  (KW)', 'Demandas Ativas Registradas Fora de Ponta -  (KW)', \
                 'Demandas Reativas Excedentes (KW)', 'Multa (R$)', 'Juros (R$)', 'TIP/CIP (R$)', \
                 'Tributos Federais (R$)', 'Total da Fatura (R$)', 'Maior Demanda Entre Ponta e Fora Ponta']]

### Calculando a do valor da fatura de energia original sem TIP, multas, juros e tributos

In [ ]:
df_cortado['Fatura Original (R$)'] = df_cortado['Total da Fatura (R$)'] - (df_cortado['Multa (R$)'] + \
                                                                           df_cortado['Juros (R$)'] + \
                                                                           df_cortado['TIP/CIP (R$)'])

### Gerando cópia do conjunto de dados filtrado

In [ ]:
df_calculo = df_cortado.copy()

---

### Criando funções que calculam ultrapassagem de demanda ativa registradas na ponta e fora da ponta
###### Considerando 5% de tolerância conforme REN 1000/2021 ANEEL

In [ ]:
def ultrapassagem_ponta(df_calculo):  
    if ((df_calculo['Demandas Ativas Registradas Na Ponta -  (KW)'] - (df_calculo['Demdas. Ativas Contdas. (DC)']*1.05)) < 0):
        return 0
    else:
        return (df_calculo['Demandas Ativas Registradas Na Ponta -  (KW)'] - df_calculo['Demdas. Ativas Contdas. (DC)'])

In [ ]:
def ultrapassagem_fora_ponta(df_calculo):  
    if ((df_calculo['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - (df_calculo['Demdas. Ativas Contdas. (DC)']*1.05)) < 0):
        return 0
    else:
        return (df_calculo['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - df_calculo['Demdas. Ativas Contdas. (DC)'])

In [ ]:
def ultrapassagem_max_ponta(df_leitura):  
    if ((df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'] - (df_leitura['maior_demanda_ativa_na_ponta']*1.05)) < 0):
        return 0
    else:
        return (df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'] - df_leitura['maior_demanda_ativa_na_ponta'])

In [ ]:
def ultrapassagem_max_fora_ponta(df_leitura):  
    if ((df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - (df_leitura['maior_demanda_ativa_fora_ponta']*1.05)) < 0):
        return 0
    else:
        return (df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - df_leitura['maior_demanda_ativa_fora_ponta'])

In [ ]:
def ultrapassagem_otima_ponta(df_leitura):  
    if ((df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'] - (df_leitura['demanda_teste']*1.05)) < 0):
        return 0
    else:
        return (df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'] - df_leitura['demanda_teste'])

In [ ]:
def ultrapassagem_otima_fora_ponta(df_leitura):  
    if ((df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - (df_leitura['demanda_teste']*1.05)) < 0):
        return 0
    else:
        return (df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'] - df_leitura['demanda_teste'])

---

### Calculando o valor das faturas considerando as modalidades tarifárias do Grupo A 
######  A demanda utilizada no cáculo foi a Demanda Contratada vigente do mês

In [ ]:
# realizei o cálculo com  'Consumos Reativos (KWh)' pois 'Cons. Reativ. Exced. Regist. (RCRE)' tinha alguns valores estranhos na tabela

for tipo_tarifa in tarifa_a4.index: 
    
    df_calculo['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] = \
    df_calculo['Consumos Ativos Na Ponta (KWh)']*tarifa_a4.at[tipo_tarifa, 'Consumo Ativo na Ponta']
    
    df_calculo['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] = \
    df_calculo['Consumos Ativos Fora de Ponta (KWh)']*tarifa_a4.at[tipo_tarifa, 'Consumo Ativo Fora Ponta']
    
    df_calculo['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] = \
    df_calculo['Consumos Reativos (KWh)']*tarifa_a4.at[tipo_tarifa, 'Consumo Reativo Excedente']
    
    df_calculo['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] = \
    df_calculo['Demdas. Ativas Contdas. (DC)']*tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']
    
    df_calculo['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] = \
    df_calculo['Demdas. Ativas Contdas. (DC)']*tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']
    
    df_calculo['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] = \
    df_calculo['Demandas Reativas Excedentes (KW)']*tarifa_a4.at[tipo_tarifa, 'Demanda Reativa Excedente na Ponta e Fora Ponta']

    df_calculo['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] = \
    df_calculo.apply(lambda df_calculo: ultrapassagem_ponta(df_calculo), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']
    
    df_calculo['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa] = \
    df_calculo.apply(lambda df_calculo: ultrapassagem_fora_ponta(df_calculo), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

    df_calculo['Tarifa (R$) '+tipo_tarifa] = (df_calculo['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] + \
                                              df_calculo['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa])/tributos

### Calculando valor das faturas considerando a classificação tarifária Grupo B3

In [ ]:
df_calculo['Tarifa (R$) Consumo Ativo - B3'] = \
df_calculo['Consumos Ativos (KWh)'] * tarifa_b3.at['B3', 'Consumo Ativo']

df_calculo['Tarifa (R$) Consumo Reativo Excedente - B3'] = \
df_calculo['Consumos Reativos (KWh)'] * tarifa_b3.at['B3', 'Consumo Reativo Excedente']

df_calculo['Tarifa (R$) B3'] = (df_calculo['Tarifa (R$) Consumo Ativo - B3'] + \
                                df_calculo['Tarifa (R$) Consumo Reativo Excedente - B3'] ) / tributos

----

# - - - - - GRÁFICOS COMPARATIVOS DE CONSUMO - - - - -

---

# Consumo Ativo Total, Ponta e Fora Ponta

In [ ]:
# Apêndice D

plt.rc('figure', figsize = (10,1.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()
contador = 0
for i in unidades_set22_grupo_a_sort:
    df_leitura = df[df['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= '2019-01']   
    df_grafico_1 = df_leitura[['Consumos Ativos (KWh)']]
    df_grafico_2 = df_leitura[['Consumos Ativos Fora de Ponta (KWh)', 'Consumos Ativos Na Ponta (KWh)']]

    ax = df_grafico_1.plot(kind='bar', color='blue', alpha=0.25, width = .8)
    df_grafico_2.plot(kind='line', ax=ax, color=('green','tomato'), lw=1.5)
   
    plt.xticks(rotation=90)
    plt.legend(['Consumo ativo fora de ponta', 'Consumo ativo na ponta', 'Consumo ativo total'], fontsize = 8)       
    plt.title(i, {'fontsize': 12}, fontweight="bold")
    plt.xlabel("", {'fontsize': 10})
    plt.ylabel("kWh", {'fontsize': 10})
    contador = contador + 1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
    plt.show()

---

# Demanda Contrada Vigente, e Demandas Registradas Ponta e Fora Ponta

In [ ]:
# Apendice C
# Aqui conseguimos visualizar unidades que possuem muita discrepância entre a demanda contratada e a utilizada

plt.rc('figure', figsize = (10,1.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

contador = 0
unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()
for i in unidades_set22_grupo_a_sort:
    
    df_leitura = df[df['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= '2017-09']
    df_leitura1 = df_leitura[['Demdas. Ativas Contdas. (DC)']]
    df_leitura2 = df_leitura[['Demandas Ativas Registradas Fora de Ponta -  (KW)', 'Demandas Ativas Registradas Na Ponta -  (KW)']]

    fig = df_leitura1.plot(kind='bar', color='olive', alpha=0.3, width = .8)
    df_leitura2.plot(kind='line', ax=fig, color=('green','red'), lw=3)
    
    plt.xticks(rotation=90)
    plt.legend(['D. fora ponta', 'D. na ponta', 'D. contratada'], fontsize = 8)          
    plt.title(i, {'fontsize': 12}, fontweight="bold")
    plt.xlabel("", {'fontsize': 10})
    plt.ylabel("kW", {'fontsize': 10})
    contador = contador + 1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
    plt.show()

---

# Unidades com demanda registrada menor que 75 kW desde 2019
# Demanda Contrada Vigente, e Demandas Registradas Ponta e Fora Ponta

In [ ]:
# Apêndice B
# Unidades que poderiam ser Grupo B e não pagar tarifa de Demanda

plt.rc('figure', figsize = (10,1.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

contador = 0
unidades_set22_grupo_a_75menos_sort = unidades_set22_grupo_a_75menos.copy()
unidades_set22_grupo_a_75menos_sort.sort()
for i in unidades_set22_grupo_a_75menos_sort:
    
    df_leitura = df[df['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= '2019-01']   
    df_leitura1 = df_leitura[['Demdas. Ativas Contdas. (DC)']]
    df_leitura2 = df_leitura[['Demandas Ativas Registradas Na Ponta -  (KW)', 'Demandas Ativas Registradas Fora de Ponta -  (KW)']]

    ax = df_leitura1.plot(kind='bar', color='gray', alpha=0.3, width = .8)
    df_leitura2.plot(kind='line', ax=ax, color=('red', 'green'), lw=1.5)

    plt.axhline(y=75, color = 'purple', lw=2, alpha=0.5, label='75 kW')
   
    plt.xticks(rotation=90)
    plt.legend(['D. ativa na ponta', 'D. ativa fora de ponta', '75 kW', 'D. ativa contratada'], fontsize = 8)          
    plt.title(i, {'fontsize': 12}, fontweight="bold")
    plt.xlabel("", {'fontsize': 10})
    plt.ylabel("kW", {'fontsize': 10})
    contador = contador + 1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
    plt.show()

---

# Média móvel (12 meses) da Demanda Registrada 
# Demanda Contratada ao fundo

In [ ]:
# Pensei em utilizar essa forma para otimizar a escolha de uma nova demanda contratada para as unidades

plt.rc('figure', figsize = (15,5))

for i in unidades_set22_grupo_a:
    
    df_leitura = df[df['Unidade/UFBA'] == i]
    
    df_leitura['Média Móvel Demanda Registrada'] = df_leitura['Maior Demanda Entre Ponta e Fora Ponta'].rolling(12).mean() # Média móvel 12 meses Demanda

    sns.barplot(y = 'Demdas. Ativas Contdas. (DC)', 
                x = df_leitura.index, 
                data = df_leitura,
                color = 'orange', 
                alpha = 0.3
               ).set_title(i)

    sns.pointplot(y = 'Média Móvel Demanda Registrada', 
                  x = df_leitura.index, 
                  data = df_leitura, 
                  color = 'green',
                  markers = '',
                  label='Média Móvel 12 meses Demanda Registrada'
                 )

    plt.xticks(rotation=90)
    plt.show()

---

# - - - - - GRÁFICOS COMPARATIVOS COM VALORES - - - - -

---

# Fatura Original x Tarifa Grupo A4 Verde

In [ ]:
# Apêndice A
# Conferindo se os cálculos estão sendo feitos corretamente

plt.rc('figure', figsize = (5,2))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
contador = 0

unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()
for i in unidades_set22_grupo_a_sort:
    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= '2022-05']
    
    df_leitura = df_leitura[['Tarifa (R$) A4 VERDE 2022', 'Fatura Original (R$)']]
    
    df_leitura.plot.bar(color =['darkgreen', 'darkorange'], width = 0.7)

    plt.xticks(rotation=0)
    plt.legend(['Calculado', 'Original'], fontsize = 8)    
    plt.title(i, {'fontsize': 10}, fontweight="bold")
    plt.xlabel("", {'fontsize': 10})
    plt.ylabel("R$", {'fontsize': 10})
    
    contador=contador+1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
    plt.show()    

---

# Tarifa VERDE vs AZUL
# Demanda contratada vigente

In [ ]:
# A ultrapassagem de demanda foi devidamente calculada 
# Aqui considerei a demanda contratada na ponta AZUL a mesma que a fora ponta, então os valores da azul ficaram bem altos

plt.rc('figure', figsize = (15,5))

mes_inicial = '2022-05'

for i in unidades_set22_grupo_a:

    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= mes_inicial]
    df_leitura = df_leitura[['Tarifa (R$) A4 AZUL 2022', 'Tarifa (R$) A4 VERDE 2022']] #, 'Fatura Original (R$)']]

    fig = df_leitura.plot.bar(color =['blue', 'green']) # , 'gold'])

    plt.xticks(rotation=0)
    fig.set_ylabel('Valor (R$)', {'fontsize': 14})
    fig.set_xlabel('Data', {'fontsize': 14})
    fig.set_title(i, {'fontsize': 18})
    plt.legend()
    plt.show()
    

---

# Tarifa VERDE vs AZUL
# Considerando como contratada a demanda máxima que aconteceu nos últimos 12 meses acrescida de 10%

In [ ]:
# Nessa versão a tarifa AZUL ficou mais suave mas ainda sim não se demonstrou viável em relação a VERDE

plt.rc('figure', figsize = (15,5))

mes_maxima_demanda = '2019-09' 
mes_escolhido = '2022-05'

for i in unidades_set22_grupo_a:
    
    df_leitura = df_cortado[df_cortado['Unidade/UFBA'] == i]
   
    #Salvar maior demanda do último ano e aumentar 10%
    df_leitura = df_leitura[df_leitura.index >= mes_maxima_demanda]
    
    df_leitura['maior_demanda_ativa_na_ponta'] = \
    df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'].max() * 1.1
    
    df_leitura['maior_demanda_ativa_fora_ponta'] = \
    df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'].max() * 1.1    
   
    for tipo_tarifa in tarifa_a4.index:

        df_leitura['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] = \
        df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo na Ponta']
        
        df_leitura['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] = \
        df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo Fora Ponta']
        
        df_leitura['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] = \
        df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Reativo Excedente']
        
        df_leitura['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] = \
        df_leitura['maior_demanda_ativa_na_ponta'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']
        
        df_leitura['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] = \
        df_leitura['maior_demanda_ativa_fora_ponta'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']
        
        df_leitura['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] = \
        df_leitura['Demandas Reativas Excedentes (KW)']*tarifa_a4.at[tipo_tarifa, 'Demanda Reativa Excedente na Ponta e Fora Ponta']
       
        df_leitura['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_max_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']
        
        df_leitura['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_max_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

        df_leitura['Tarifa (R$) '+tipo_tarifa] = (df_leitura['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos
        
    # As tarifas mudaram o preço em maio 2022
    df_grafico = df_leitura[df_leitura.index >= mes_escolhido]
    df_grafico = df_grafico[['Tarifa (R$) A4 AZUL 2022', 'Tarifa (R$) A4 VERDE 2022']]
    
    fig = df_grafico.plot.bar(color =['blue', 'green'])

    plt.xticks(rotation=0)
    fig.set_ylabel('Valor (R$)', {'fontsize': 14})
    fig.set_xlabel('Data', {'fontsize': 14})
    fig.set_title(i, {'fontsize': 18})
    plt.legend()
    plt.show()
    

---

# Tarifa VERDE vs AZUL
# Considerando como Demanda Contratada o máximo valor da média móvel da demanda que aconteceu desde 2019 acrescida de 10%

In [ ]:
plt.rc('figure', figsize = (15,5))

mes_maxima_demanda = '2021-09' 
mes_escolhido = '2022-05'

for i in unidades_set22_grupo_a:
    
    df_leitura = df_cortado[df_cortado['Unidade/UFBA'] == i]
   
    #Salvar maior demanda do último ano e aumentar 10%
    df_leitura = df_leitura[df_leitura.index >= mes_maxima_demanda]
    
#     df_leitura['Média Móvel Demanda Registrada'] = df_leitura['Maior Demanda Entre Ponta e Fora Ponta'].rolling(12).mean() # Média móvel 12 meses Demanda
#     df_leitura['maior_demanda_ativa'] = df_leitura['Média Móvel Demanda Registrada'].max() * 1.1

#     df_leitura['maior_demanda_ativa_na_ponta'] = df_leitura['maior_demanda_ativa']
#     df_leitura['maior_demanda_ativa_fora_ponta'] = df_leitura['maior_demanda_ativa']
    
    df_leitura['maior_demanda_ativa_na_ponta'] = \
    df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'].rolling(12).mean() # Média móvel 12 meses Demanda
    
    df_leitura['maior_demanda_ativa_na_ponta'] = \
    df_leitura['Demandas Ativas Registradas Na Ponta -  (KW)'].max() * 1.1 # Pega o maior valor da média móvel no período selecionado
    
    df_leitura['maior_demanda_ativa_fora_ponta'] = \
    df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'].rolling(12).mean() # Média móvel 12 meses Demanda   
    
    df_leitura['maior_demanda_ativa_fora_ponta'] = \
    df_leitura['Demandas Ativas Registradas Fora de Ponta -  (KW)'].max() * 1.1 # Pega o maior valor da média móvel no período selecionado    
   
    for tipo_tarifa in tarifa_a4.index:

        df_leitura['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] = df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo na Ponta']
        df_leitura['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] = df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo Fora Ponta']
        df_leitura['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] = df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Reativo Excedente']
        df_leitura['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] = df_leitura['maior_demanda_ativa_na_ponta'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']
        df_leitura['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] = df_leitura['maior_demanda_ativa_fora_ponta'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']
        
        df_leitura['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] = df_leitura['Demandas Reativas Excedentes (KW)'] \
        * tarifa_a4.at[tipo_tarifa, 'Demanda Reativa Excedente na Ponta e Fora Ponta']
       
        df_leitura['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] = df_leitura.apply(lambda df_leitura: ultrapassagem_max_ponta(df_leitura), axis=1) \
        * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']
        
        df_leitura['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa] = df_leitura.apply(lambda df_leitura: ultrapassagem_max_fora_ponta(df_leitura), axis=1) \
        * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

        df_leitura['Tarifa (R$) '+tipo_tarifa] = (df_leitura['Tarifa (R$) Consumo Ativo na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Consumo Ativo Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Consumo Reativo Excedente '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Ativa na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Ultrapassagem na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Tarifa (R$) Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos
        
    # As tarifas mudaram o preço em maio 2022
    df_grafico = df_leitura[df_leitura.index >= mes_escolhido]
    df_grafico = df_grafico[['Tarifa (R$) A4 AZUL 2022', 'Tarifa (R$) A4 VERDE 2022', 'Fatura Original (R$)']]
    
    fig = df_grafico.plot.bar(color =['blue', 'green', 'darkorange'])

    plt.xticks(rotation=0)
    fig.set_ylabel('Valor (R$)', {'fontsize': 14})
    fig.set_xlabel('Data', {'fontsize': 14})
    fig.set_title(i, {'fontsize': 18})
    plt.legend()
    plt.show()
    

---

# Tarifa VERDE vs FATURA ORIGINAL
# DEMANDA ÓTIMA

In [ ]:
plt.rc('figure', figsize = (15,5))

mes_demanda = '2022-05' 
mes_inicial_grafico = '2022-05'
mes_final = '2022-09'

valor_demandas_cumulativas = []
demandas_otimas = []
demandas_originais = []

for i in unidades_set22_grupo_a:
    
    j = 0
    demandas_temporarias = []
    valor_demandas_temporarias = []
    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= mes_demanda]            
    
    df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] = \
    df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Ativo na Ponta']
    
    df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] = \
    df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Ativo Fora Ponta']
    
    df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] = \
    df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Reativo Excedente']
    
    df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] = \
    df_leitura['Demandas Reativas Excedentes (KW)']*tarifa_a4.at['A4 VERDE 2022', 'Demanda Reativa Excedente na Ponta e Fora Ponta']
        
    while j < 800:

        j = j+1
        df_leitura['demanda_teste'] = j                
        
        df_leitura['Valor Demanda Ativa A4 VERDE 2022'] = \
        df_leitura['demanda_teste'] * tarifa_a4.at['A4 VERDE 2022', 'Demanda Ativa Fora Ponta']
        
        df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem na Ponta']
        
        df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022'] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem Fora Ponta']
        
        df_leitura['Valor A4 VERDE 2022 teste'] = (df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] + \
                                                   df_leitura['Valor Demanda Ativa A4 VERDE 2022'] + \
                                                   df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022']) / tributos

        valor_demandas_temporarias.append(round(df_leitura['Valor A4 VERDE 2022 teste'].mean(), 2))
        demandas_temporarias.append(j)
        
    valor_demandas_cumulativas.append(valor_demandas_temporarias)
    
    valor_minimo = min(valor_demandas_temporarias)
    indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
    demanda_otima = demandas_temporarias[indice_valor_minimo]
    
    if demanda_otima < 30:
        demanda_otima = 30
        
    demandas_otimas.append(demanda_otima)
    demandas_originais.append(int(df_leitura.at[mes_final,'Demdas. Ativas Contdas. (DC)']))
    df_leitura['demanda_teste'] = demanda_otima        
    
    df_leitura['Valor Demanda Ativa A4 VERDE 2022'] = \
    df_leitura['demanda_teste'] * tarifa_a4.at['A4 VERDE 2022', 'Demanda Ativa Fora Ponta']
    
    df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] = \
    df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem na Ponta']
    
    df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022'] = \
    df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem Fora Ponta']
    
    df_leitura['Valor A4 VERDE 2022'] = round((df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] + \
                                               df_leitura['Valor Demanda Ativa A4 VERDE 2022'] + \
                                               df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022']) / tributos, 2)
         
    df_grafico = df_leitura[df_leitura.index >= mes_inicial_grafico]
    df_grafico = df_grafico[['Valor A4 VERDE 2022', 'Tarifa (R$) A4 VERDE 2022']]
    
    fig = df_grafico.plot.bar(color =['darkgreen', 'tomato'])

    plt.xticks(rotation=0)
    fig.set_ylabel('Valor (R$)', {'fontsize': 14})
    fig.set_xlabel('Data', {'fontsize': 14})
    fig.set_title(i, {'fontsize': 18})
    plt.legend(['Valor com D. ótima', 'Valor original'])
    plt.show()
    print('Demanda ótima: '+str(demanda_otima))
    print('Demanda original: '+str(int(df_leitura.at[mes_final,'Demdas. Ativas Contdas. (DC)'])))

---

# Curva ótima do Valor da Fatura x Demanda

In [ ]:
plt.rc('figure', figsize = (10,1.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

mes_demanda = '2022-05'
contador = 0
ordenadas = list(range(0,800))

unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()

for i in unidades_set22_grupo_a_sort:
    
    valor_leitura = valor_demandas_cumulativas[contador]
    demandas_leitura = demandas_otimas[contador]
    
    fig = plt.plot(ordenadas, valor_leitura, label='Valor da fatura', marker='', lw=2)
    
    plt.axvline(x=demandas_otimas[contador], 
                color = 'purple', 
                lw=3, alpha=0.4, 
                label='D. ótima: '+str(demandas_otimas[contador])+' kW')
    plt.axvline(x=demandas_originais[contador], 
                color = 'tomato', 
                lw=3, alpha=0.4, 
                label='D. atual: '+str(demandas_originais[contador])+' kW')

    contador = contador+1   

    plt.xticks(rotation=0)      
    plt.legend()
    plt.title(i, {'fontsize': 12}, fontweight="bold")
    plt.xlabel("Demanda (kW)", {'fontsize': 10})
    plt.ylabel("R$", {'fontsize': 10})
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
    plt.show()  


---

# VERDE vs AZUL
# DEMANDA ÓTIMA

In [ ]:
plt.rc('figure', figsize = (10,1.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
contador = 0

mes_demanda = '2022-05' 
mes_inicial_grafico = '2022-05'
mes_final = '2022-09'

unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()

for i in unidades_set22_grupo_a_sort:
    
    demanda_otima_verde = []
    demanda_otima_azul_fora_ponta = []
    demanda_otima_azul_na_ponta = []
    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= mes_demanda] 
        
    for tipo_tarifa in tarifa_a4.index:
             
        if tipo_tarifa == 'A4 VERDE 2022':
            
            demandas_temporarias = []
            valor_demandas_temporarias = []
            j = 0
            while j < 800:

                j = j+1
                df_leitura['demanda_teste'] = j                

                df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
                df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

                df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
                df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

                df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
                df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

                df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] +  \
                                                       df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] + \
                                                       df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

                valor_demandas_temporarias.append(round(df_leitura['Valor_1 '+tipo_tarifa].mean(), 2))
                demandas_temporarias.append(j)

            valor_minimo = min(valor_demandas_temporarias)
            indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
            demanda_otima = demandas_temporarias[indice_valor_minimo]
            
            if demanda_otima < 30:
                demanda_otima = 30

            demanda_otima_verde = str(demanda_otima)            
            df_leitura['demanda_teste'] = demanda_otima        

            df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

            df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

            df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

            df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] +  \
                                                   df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] + \
                                                   df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

        if tipo_tarifa == 'A4 AZUL 2022':
        
            demandas_temporarias = []
            valor_demandas_temporarias = []        
            j = 0
            while j < 800:

                j = j+1
                df_leitura['demanda_teste'] = j                
                
                df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
                df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

                df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
                df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']
                
                df_leitura['Valor_2 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                                      df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos

                valor_demandas_temporarias.append(round(df_leitura['Valor_2 '+tipo_tarifa].mean(), 2))
                demandas_temporarias.append(j)

            valor_minimo = min(valor_demandas_temporarias)
            indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
            demanda_otima = demandas_temporarias[indice_valor_minimo]

            if demanda_otima < 30:
                demanda_otima = 30            

            demanda_otima_azul_fora_ponta = str(demanda_otima)            
            df_leitura['demanda_teste'] = demanda_otima        

            df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

            df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

            df_leitura['Valor_2 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos

            demandas_temporarias = []
            valor_demandas_temporarias = []        
            j = 0
            while j < 800:

                j = j+1
                df_leitura['demanda_teste'] = j                
                
                df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] = \
                df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']
                
                df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
                df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']
                
                df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] + \
                                                      df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

                valor_demandas_temporarias.append(round(df_leitura['Valor_1 '+tipo_tarifa].mean(), 2))
                demandas_temporarias.append(j)

            valor_minimo = min(valor_demandas_temporarias)
            indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
            demanda_otima = demandas_temporarias[indice_valor_minimo]
            
            if demanda_otima < 30:
                demanda_otima = 30

            demanda_otima_azul_na_ponta = str(demanda_otima)            
            df_leitura['demanda_teste'] = demanda_otima        

            df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']

            df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

            df_leitura['Valor_1 '+tipo_tarifa] = ((df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] + \
                                                   df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos) +  \
                                                   df_leitura['Valor_2 '+tipo_tarifa]                   
                
        df_leitura['Valor Consumo Ativo na Ponta '+tipo_tarifa] = \
        df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo na Ponta']
        
        df_leitura['Valor Consumo Ativo Fora Ponta '+tipo_tarifa] = \
        df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo Fora Ponta']
        
        df_leitura['Valor Consumo Reativo Excedente '+tipo_tarifa] = \
        df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Reativo Excedente']
        
        df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] = \
        df_leitura['Demandas Reativas Excedentes (KW)']*tarifa_a4.at[tipo_tarifa, 'Demanda Reativa Excedente na Ponta e Fora Ponta']

        df_leitura['Valor '+tipo_tarifa] = round((df_leitura['Valor_1 '+tipo_tarifa] + \
                                                  df_leitura['Valor Consumo Ativo na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Valor Consumo Ativo Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Valor Consumo Reativo Excedente '+tipo_tarifa] +  \
                                                  df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa]) / tributos, 2)
        
    # As tarifas mudaram o preço em maio 2022
    df_grafico = df_leitura[df_leitura.index >= mes_inicial_grafico]
    df_grafico = df_grafico[['Valor A4 VERDE 2022', 'Valor A4 AZUL 2022', 'Tarifa (R$) A4 VERDE 2022']]
    
    fig = df_grafico.plot.bar(color =['darkgreen', 'darkblue', 'darkorange'])
    
    demanda_original = str(int(df_leitura.at[mes_final,'Demdas. Ativas Contdas. (DC)']))
    
    plt.xticks(rotation=0)
    plt.legend(['Valor A4 Verde', 'Valor A4 Azul', 'Valor Original'], fontsize = 8)
    fig.set_ylabel('R$', {'fontsize': 10})
    fig.set_xlabel('Demanda ótima Verde e fora ponta Azul: '+demanda_otima_verde+ \
                   ' kW | Demanda ótima na ponta Azul: '+demanda_otima_azul_na_ponta+ \
                   ' kW | Demanda original: '+demanda_original+ \
                   ' kW', {'fontsize': 10})
    plt.title(i, {'fontsize': 10}, fontweight="bold")
    
    contador=contador+1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
    plt.show()   
   
    print('Demanda ótima VERDE: '+demanda_otima_verde)
    print('Demanda ótima fora ponta AZUL: '+demanda_otima_azul_fora_ponta)
    print('Demanda ótima na ponta AZUL: '+demanda_otima_azul_na_ponta)    
    print('Demanda original: '+demanda_original)

---

# Tarifa B3 vs Fatura Original

In [ ]:
# Conferindo se os cálculos estão sendo feitos corretamente. Há uma pequena diferença mas não parece ser erro de cálculo, e sim alguma taxa ou dívida

plt.rc('figure', figsize = (15,5))

for i in unidades_set22_grupo_b:
    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= '2022-05']
    
    df_leitura = df_leitura[['Tarifa (R$) B3', 'Fatura Original (R$)']]
    
    fig = df_leitura.plot.bar(color =['darkcyan', 'darkorange'])

    plt.xticks(rotation=0)
    fig.set_ylabel('Valor (R$)', {'fontsize': 14})
    fig.set_xlabel('Data', {'fontsize': 14})
    fig.set_title(i, {'fontsize': 18})
    plt.legend()
    plt.show()

---

# Valor Fatura B3 vs A4 VERDE
# Somente unidades com menos de 75kW de demanda registrada desde jan2019 

In [ ]:
# Apêndice E

plt.rc('figure', figsize = (5,2.5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

contador = 0

unidades_set22_grupo_a_75menos_sort = unidades_set22_grupo_a_75menos.copy()
unidades_set22_grupo_a_75menos_sort.sort()
for i in unidades_set22_grupo_a_75menos_sort:
        
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= '2022-05']            
    df_leitura = df_leitura[['Tarifa (R$) B3', 'Tarifa (R$) A4 VERDE 2022']]
    
    fig = df_leitura.plot.bar(color =['darkorange', 'darkgreen'], width = 0.7, label=['B3', 'A4 VERDE 2022'])   


    plt.xticks(rotation=0)
    plt.legend(['Valor B3', 'Valor A4'], fontsize = 8)    
    plt.title(i, {'fontsize': 10}, fontweight="bold")
    plt.xlabel("", {'fontsize': 10})
    plt.ylabel("R$", {'fontsize': 10})
    contador=contador+1
#     plt.savefig(str(contador)+'.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
    plt.show()

---

### Histórico de faturas do Grupo B

In [ ]:
plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

unidades_set22_grupo_b_sort = unidades_set22_grupo_b.copy()
unidades_set22_grupo_b_sort.sort()
for i in unidades_set22_grupo_b_sort:
        
    df_leitura = df[df['Unidade/UFBA'] == i]    
#     df_leitura = df_leitura[df_leitura.index >= '2022-05']            
    df_leitura = df_leitura[['Consu. Ativos  Regist. (CR)']]
    
    fig = df_leitura.plot.bar(color =['darkorange'], width = 0.7, label=['B3'])   

    plt.xticks(rotation=90)
    plt.title(i, {'fontsize': 10}, fontweight="bold")
    plt.show()

---

# - - - - - ECONOMIA - - - - -

---

# Tabela de valores ao mudar de A4 VERDE para B3
# Unidades que não passam de 75KWh/mês desde jan2019

In [ ]:
df_valores_economizados_grupo_a_vs_b = pd.DataFrame()
economia_acumulada = 0
contador=0

mes_escolhido = '2022-05'

unidades_set22_grupo_a_75menos_sort = unidades_set22_grupo_a_75menos.copy()
unidades_set22_grupo_a_75menos_sort.sort()
for i in unidades_set22_grupo_a_75menos_sort:
    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
    df_leitura = df_leitura[df_leitura.index >= mes_escolhido]    
        
    total_tarifa_b3 = df_leitura['Tarifa (R$) B3'].sum()    
    total_tarifa_a4 = df_leitura['Tarifa (R$) A4 VERDE 2022'].sum()\
    
    if total_tarifa_b3 < total_tarifa_a4:            
        
        economia_valor =  total_tarifa_a4 - total_tarifa_b3
        economia_valor = round(economia_valor,2)
        
        economia_porcentagem = -100*(total_tarifa_b3 - total_tarifa_a4)/total_tarifa_a4
        economia_porcentagem = round(economia_porcentagem,2)
        
        economia_acumulada = economia_acumulada + economia_valor
               
        df_valores_economizados_grupo_a_vs_b.at[i,'Economia Total (%)'] = economia_porcentagem
        df_valores_economizados_grupo_a_vs_b.at[i,'Economia Total (R$)'] = economia_valor
        df_valores_economizados_grupo_a_vs_b.at[i,'Economia por mês (R$)'] = round(economia_valor/5,2)
                
economia_acumulada = round(economia_acumulada,2)
print()
print('- - - - - - - - - - - - - - - - - - - - -')
print('Total economizado no período: R$ '+str(economia_acumulada))
print('- - - - - - - - - - - - - - - - - - - - -')

df_valores_economizados_grupo_a_vs_b

### Exportando para XSLX a tabela dos valores economizados ao mudar de A4 VERDE para B3 

In [ ]:
# df_valores_economizados_grupo_a_vs_b.to_excel('df_valores_economizados_grupo_a_vs_b.xlsx')

---

# Tabela de valores se a demanda contratada fosse a DEMANDA ÓTIMA
# Considerando o mínimo contratado de 30 kW

In [ ]:
df_valores_economizados_demanda = pd.DataFrame()
economia_acumulada = 0
contador=0

mes_demanda = '2022-05' 
mes_inicial = '2022-05'
mes_final = '2022-09'

unidades_set22_grupo_a_sort = unidades_set22_grupo_a.copy()
unidades_set22_grupo_a_sort.sort()

for i in unidades_set22_grupo_a_sort:
    
    j = 0
    demandas_temporarias = []
    valor_demandas_temporarias = []    
    df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]
    df_leitura = df_leitura[df_leitura.index >= mes_demanda]            
    
    df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] = \
    df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Ativo na Ponta']
    
    df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] = \
    df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Ativo Fora Ponta']
    
    df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] = \
    df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at['A4 VERDE 2022', 'Consumo Reativo Excedente']
    
    df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] = \
    df_leitura['Demandas Reativas Excedentes (KW)']*tarifa_a4.at['A4 VERDE 2022', 'Demanda Reativa Excedente na Ponta e Fora Ponta']
        
    while j < 800:

        j = j+1
        df_leitura['demanda_teste'] = j                
        
        df_leitura['Valor Demanda Ativa A4 VERDE 2022'] = \
        df_leitura['demanda_teste'] * tarifa_a4.at['A4 VERDE 2022', 'Demanda Ativa Fora Ponta']
        
        df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem na Ponta']
        
        df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022'] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem Fora Ponta']
        
        df_leitura['Valor A4 VERDE 2022 teste'] = (df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] + \
                                                   df_leitura['Valor Demanda Ativa A4 VERDE 2022'] + \
                                                   df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] + \
                                                   df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022']) / tributos

        valor_demandas_temporarias.append(round(df_leitura['Valor A4 VERDE 2022 teste'].mean(), 2))
        demandas_temporarias.append(j)
            
    valor_minimo = min(valor_demandas_temporarias)
    indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
    demanda_otima = demandas_temporarias[indice_valor_minimo]
    
    if demanda_otima < 30:
        demanda_otima = 30
        
    df_leitura['demanda_teste'] = demanda_otima        
    df_leitura['Valor Demanda Ativa A4 VERDE 2022'] = \
    df_leitura['demanda_teste'] * tarifa_a4.at['A4 VERDE 2022', 'Demanda Ativa Fora Ponta']
    
    df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] = \
    df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * \
    tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem na Ponta']
    
    df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022'] = \
    df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * \
    tarifa_a4.at['A4 VERDE 2022', 'Ultrapassagem Fora Ponta']
    
    df_leitura['Valor A4 VERDE 2022'] = round((df_leitura['Valor Consumo Ativo na Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Consumo Ativo Fora Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Consumo Reativo Excedente A4 VERDE 2022'] + \
                                               df_leitura['Valor Demanda Ativa A4 VERDE 2022'] + \
                                               df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Ultrapassagem na Ponta A4 VERDE 2022'] + \
                                               df_leitura['Valor Ultrapassagem Fora Ponta A4 VERDE 2022']) / tributos, 2)
         
    # As tarifas mudaram o preço em maio 2022
    df_economia = df_leitura[df_leitura.index >= mes_inicial]
    
    total_tarifa_original = df_economia['Tarifa (R$) A4 VERDE 2022'].sum()       
    total_tarifa_a4 = df_economia['Valor A4 VERDE 2022'].sum()
        
    economia_valor =  total_tarifa_original - total_tarifa_a4
    economia_valor = round(economia_valor,2)

    economia_porcentagem = -100*(total_tarifa_a4 - total_tarifa_original)/total_tarifa_original
    economia_porcentagem = round(economia_porcentagem,2)

    economia_acumulada = economia_acumulada + economia_valor

    demanda_atual = int(df_leitura.at[mes_final,'Demdas. Ativas Contdas. (DC)'])
        
    df_valores_economizados_demanda.at[i,'Demanda Ótima (kW)'] = demanda_otima
    df_valores_economizados_demanda.at[i,'Demanda Atual (kW)'] = demanda_atual
    df_valores_economizados_demanda.at[i,'Economia Total (%)'] = economia_porcentagem
    df_valores_economizados_demanda.at[i,'Economia Total (R$)'] = economia_valor
    df_valores_economizados_demanda.at[i,'Economia por mês (R$)'] = round(economia_valor/5,2)
        
economia_acumulada = round(economia_acumulada,2)
print('- - - - - - - - - - - - - - - - - - - -')
print('Total economizado no período: R$ '+str(economia_acumulada))
print('- - - - - - - - - - - - - - - - - - - -')
df_valores_economizados_demanda

### Exportando para XSLX tabela dos valores economizados com a DEMANDA ÓTIMA

In [ ]:
# df_valores_economizados_demanda.to_excel('df_valores_economizados_demanda.xlsx')

---

### GRUPO A - VERDE para AZUL
#### Quanto economizaria se mudasse o grupo tarifário para AZUL

Não economizaria

---

### Criando Figuras para o texto

In [ ]:
# Figura 2
# Fatura calculada vs Fatura Oritinal
# BEX – Biblioteca Universitária de Exatas Omar Catunda

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
contador = 0

i = 'BEX - Biblioteca Universitária de Exatas Omar Catunda'
    
df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
df_leitura = df_leitura[df_leitura.index >= '2022-05']

df_leitura = df_leitura[['Tarifa (R$) A4 VERDE 2022', 'Fatura Original (R$)']]

df_leitura.plot.bar(color =['darkgreen', 'darkorange'], width = 0.7)

plt.xticks(rotation=0)
plt.legend(['Valor calculado', 'Valor original'], fontsize = 12)
plt.xlabel("Mês/ano", {'fontsize': 12})
plt.ylabel("R$", {'fontsize': 12})

contador=contador+1
# plt.savefig('Figuras\Figura 2.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
plt.show()    

In [ ]:
# Figura 3
# Demanda contratada e registrada. 75 kW ou menos
# Unidade que poderia ser Grupo B

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

contador = 0

i = 'CEAB - FACULDADE DE ARQUITETURA E URBANISMO'
    
df_leitura = df[df['Unidade/UFBA'] == i]
df_leitura = df_leitura[df_leitura.index >= '2019-01']   
df_leitura1 = df_leitura[['Demdas. Ativas Contdas. (DC)']]
df_leitura2 = df_leitura[['Demandas Ativas Registradas Na Ponta -  (KW)', 'Demandas Ativas Registradas Fora de Ponta -  (KW)']]

ax = df_leitura1.plot(kind='bar', color='gray', alpha=0.3, width = .8)
df_leitura2.plot(kind='line', ax=ax, color=('red', 'green'), lw=2)

plt.axhline(y=75, color = 'purple', lw=2, alpha=0.5, label='75 kW')

plt.xticks(rotation=90)
plt.legend(['Demanda ativa na ponta', 'Demanda ativa fora de ponta', '75 kW', 'Demanda ativa contratada'], fontsize = 10)          
plt.xlabel("Mês/ano", {'fontsize': 12})
plt.ylabel("kW", {'fontsize': 12})
contador = contador + 1
# plt.savefig('Figuras\Figura 3.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
plt.show()

In [ ]:
# Figura 4
# Demanda contratada e registrada.

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 10

i = 'PAF V - Pavilhão de Aulas da Federação V'

df_leitura = df[df['Unidade/UFBA'] == i]    
df_leitura = df_leitura[df_leitura.index >= '2017-09']
df_leitura1 = df_leitura[['Demdas. Ativas Contdas. (DC)']]
df_leitura2 = df_leitura[['Demandas Ativas Registradas Na Ponta -  (KW)', 'Demandas Ativas Registradas Fora de Ponta -  (KW)']]

fig = df_leitura1.plot(kind='bar', color='olive', alpha=0.3, width = .8)
df_leitura2.plot(kind='line', ax=fig, color=('red','green'), lw=3)

plt.xticks(rotation=90)
plt.legend(['Demanda ativa na ponta', 'Demanda ativa fora de ponta', 'Demanda ativa contratada'], fontsize = 10)
plt.xlabel("Mês/ano", {'fontsize': 10})
plt.ylabel("kW", {'fontsize': 10})
# plt.savefig('Figuras\Figura 4.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
plt.show()

In [ ]:
# Figura 5
# Consumo ativo ponta e fora de ponta

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

i = 'ESCOLA POLITÉCNICA'

df_leitura = df[df['Unidade/UFBA'] == i]
df_leitura = df_leitura[df_leitura.index >= '2019-01']
df_grafico_1 = df_leitura[['Consumos Ativos (KWh)']]
df_grafico_2 = df_leitura[['Consumos Ativos Fora de Ponta (KWh)', 'Consumos Ativos Na Ponta (KWh)']]

ax = df_grafico_1.plot(kind='bar', color='blue', alpha=0.25, width = .8)
df_grafico_2.plot(kind='line', ax=ax, color=('green','tomato'), lw=2)

plt.xticks(rotation=90)
plt.legend(['Consumo ativo fora de ponta', 'Consumo ativo na ponta', 'Consumo ativo total'], fontsize = 10)       
plt.xlabel("Mês/ano", {'fontsize': 10})
plt.ylabel("kWh", {'fontsize': 10})
# plt.savefig('Figuras\Figura 5.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
plt.show()

In [ ]:
# Figura 6

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
    
i = 'CEAB - FACULDADE DE ARQUITETURA E URBANISMO'
        
df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]    
df_leitura = df_leitura[df_leitura.index >= '2022-05']            
df_leitura = df_leitura[['Tarifa (R$) B3', 'Tarifa (R$) A4 VERDE 2022']] 

fig = df_leitura.plot.bar(color =['darkorange', 'darkgreen'], width = 0.7)

plt.xticks(rotation=0)
plt.legend(['Valor fatura em B3', 'Valor fatura em A4 VERDE'], fontsize = 10)
plt.xlabel("Mês/ano", {'fontsize': 10})
plt.ylabel("R$", {'fontsize': 10})
# plt.savefig('Figuras/Figura 6.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
plt.show()

In [ ]:
# Figura 7
# Curva de faturas

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

mes_demanda = '2022-05'
contador = 0
ordenadas = list(range(0,800))

i = 'PAF V - Pavilhão de Aulas da Federação V'
    
valor_leitura = valor_demandas_cumulativas[contador]
demandas_leitura = demandas_otimas[contador]

fig = plt.plot(ordenadas, valor_leitura, label='Valor da fatura', marker='', lw=2)

plt.axvline(x=demandas_otimas[contador], 
            color = 'purple', 
            lw=3, alpha=0.4, 
            label='Demanda ativa ótima: '+str(demandas_otimas[contador])+' kW')
plt.axvline(x=demandas_originais[contador], 
            color = 'tomato', 
            lw=3, alpha=0.4, 
            label='Demanda ativa contrada atual: '+str(demandas_originais[contador])+' kW')

contador = contador+1   

plt.xticks(rotation=0)      
plt.legend(loc=2)
plt.xlabel("Demanda (kW)", {'fontsize': 10})
plt.ylabel("R$", {'fontsize': 10})
# plt.savefig('Figuras/Figura 7.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w')
plt.show()  

In [ ]:
# Figura 8

plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

mes_demanda = '2022-05' 
mes_inicial_grafico = '2022-05'
mes_final = '2022-09'

i = 'ESCOLA POLITÉCNICA'
    
demanda_otima_verde = []
demanda_otima_azul_fora_ponta = []
demanda_otima_azul_na_ponta = []

df_leitura = df_calculo[df_calculo['Unidade/UFBA'] == i]
df_leitura = df_leitura[df_leitura.index >= mes_demanda] 

for tipo_tarifa in tarifa_a4.index:

    if tipo_tarifa == 'A4 VERDE 2022':

        demandas_temporarias = []
        valor_demandas_temporarias = []
        j = 0
        while j < 800:

            j = j+1
            df_leitura['demanda_teste'] = j                

            df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

            df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

            df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

            df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] +  \
                                                   df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] + \
                                                   df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

            valor_demandas_temporarias.append(round(df_leitura['Valor_1 '+tipo_tarifa].mean(), 2))
            demandas_temporarias.append(j)

        valor_minimo = min(valor_demandas_temporarias)
        indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
        demanda_otima = demandas_temporarias[indice_valor_minimo]

        if demanda_otima < 30:
            demanda_otima = 30

        demanda_otima_verde = str(demanda_otima)            
        df_leitura['demanda_teste'] = demanda_otima        

        df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
        df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

        df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

        df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

        df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] +  \
                                               df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] + \
                                               df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

    if tipo_tarifa == 'A4 AZUL 2022':

        demandas_temporarias = []
        valor_demandas_temporarias = []        
        j = 0
        while j < 800:

            j = j+1
            df_leitura['demanda_teste'] = j                

            df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

            df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

            df_leitura['Valor_2 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                                  df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos

            valor_demandas_temporarias.append(round(df_leitura['Valor_2 '+tipo_tarifa].mean(), 2))
            demandas_temporarias.append(j)

        valor_minimo = min(valor_demandas_temporarias)
        indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
        demanda_otima = demandas_temporarias[indice_valor_minimo]

        if demanda_otima < 30:
            demanda_otima = 30            

        demanda_otima_azul_fora_ponta = str(demanda_otima)            
        df_leitura['demanda_teste'] = demanda_otima        

        df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] = \
        df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa Fora Ponta']

        df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_fora_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem Fora Ponta']

        df_leitura['Valor_2 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa Fora Ponta '+tipo_tarifa] + \
                                              df_leitura['Valor Ultrapassagem Fora Ponta '+tipo_tarifa]) / tributos

        demandas_temporarias = []
        valor_demandas_temporarias = []        
        j = 0
        while j < 800:

            j = j+1
            df_leitura['demanda_teste'] = j                

            df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] = \
            df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']

            df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
            df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

            df_leitura['Valor_1 '+tipo_tarifa] = (df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] + \
                                                  df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos

            valor_demandas_temporarias.append(round(df_leitura['Valor_1 '+tipo_tarifa].mean(), 2))
            demandas_temporarias.append(j)

        valor_minimo = min(valor_demandas_temporarias)
        indice_valor_minimo = valor_demandas_temporarias.index(valor_minimo)    
        demanda_otima = demandas_temporarias[indice_valor_minimo]

        if demanda_otima < 30:
            demanda_otima = 30

        demanda_otima_azul_na_ponta = str(demanda_otima)            
        df_leitura['demanda_teste'] = demanda_otima        

        df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] = \
        df_leitura['demanda_teste'] * tarifa_a4.at[tipo_tarifa, 'Demanda Ativa na Ponta']

        df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa] = \
        df_leitura.apply(lambda df_leitura: ultrapassagem_otima_ponta(df_leitura), axis=1) * tarifa_a4.at[tipo_tarifa, 'Ultrapassagem na Ponta']

        df_leitura['Valor_1 '+tipo_tarifa] = ((df_leitura['Valor Demanda Ativa na Ponta '+tipo_tarifa] + \
                                               df_leitura['Valor Ultrapassagem na Ponta '+tipo_tarifa]) / tributos) +  \
                                               df_leitura['Valor_2 '+tipo_tarifa]                   

    df_leitura['Valor Consumo Ativo na Ponta '+tipo_tarifa] = \
    df_leitura['Consumos Ativos Na Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo na Ponta']

    df_leitura['Valor Consumo Ativo Fora Ponta '+tipo_tarifa] = \
    df_leitura['Consumos Ativos Fora de Ponta (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Ativo Fora Ponta']

    df_leitura['Valor Consumo Reativo Excedente '+tipo_tarifa] = \
    df_leitura['Consumos Reativos (KWh)'] * tarifa_a4.at[tipo_tarifa, 'Consumo Reativo Excedente']

    df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa] = \
    df_leitura['Demandas Reativas Excedentes (KW)']*tarifa_a4.at[tipo_tarifa, 'Demanda Reativa Excedente na Ponta e Fora Ponta']

    df_leitura['Valor '+tipo_tarifa] = round((df_leitura['Valor_1 '+tipo_tarifa] + \
                                              df_leitura['Valor Consumo Ativo na Ponta '+tipo_tarifa] + \
                                              df_leitura['Valor Consumo Ativo Fora Ponta '+tipo_tarifa] + \
                                              df_leitura['Valor Consumo Reativo Excedente '+tipo_tarifa] +  \
                                              df_leitura['Valor Demanda Reativa Excedente na Ponta e Fora Ponta '+tipo_tarifa]) / tributos, 2)

# As tarifas mudaram o preço em maio 2022
df_grafico = df_leitura[df_leitura.index >= mes_inicial_grafico]
df_grafico = df_grafico[['Valor A4 VERDE 2022', 'Valor A4 AZUL 2022', 'Tarifa (R$) A4 VERDE 2022']]

fig = df_grafico.plot.bar(color =['darkgreen', 'darkblue', 'darkorange'])

demanda_original = str(int(df_leitura.at[mes_final,'Demdas. Ativas Contdas. (DC)']))

plt.xticks(rotation=0)
plt.legend(['Valor A4 Verde (demanda contratada otimizada)', \
            'Valor A4 Azul (demandas contratadas otimizadas)', \
            'Valor Original'], \
           fontsize = 10, \
           loc='lower right')
fig.set_ylabel('R$', {'fontsize': 10})
fig.set_xlabel('Demanda ótima Verde e fora ponta Azul: '+demanda_otima_verde+ \
               ' kW | Demanda ótima na ponta Azul: '+demanda_otima_azul_na_ponta+ \
               ' kW | Demanda original: '+demanda_original+ \
               ' kW', {'fontsize': 10})

# plt.savefig('Figuras/Figura 8.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
plt.show()   

print('Demanda ótima VERDE: '+demanda_otima_verde)
print('Demanda ótima fora ponta AZUL: '+demanda_otima_azul_fora_ponta)
print('Demanda ótima na ponta AZUL: '+demanda_otima_azul_na_ponta)    
print('Demanda original: '+demanda_original)

In [ ]:
# Figura 9
plt.rc('figure', figsize = (10,5))   
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8

df_leitura = pd.DataFrame()
df_leitura['Atraso'] = df['Multa (R$)'] + df['Juros (R$)']
df_grafico = df_leitura.groupby('Mês/ano').sum()

df_grafico.plot.bar(color = 'darkcyan', width = 0.85)

plt.xticks(rotation=90)
plt.legend(['Soma do valor de multas e juros'], fontsize = 10)
plt.xlabel("Mês/ano", {'fontsize': 10})
plt.ylabel("R$", {'fontsize': 10})
# plt.savefig('Figuras/Figura 9.png', pad_inches=0.1, bbox_inches ='tight', facecolor ='w') 
plt.show()